In [7]:
!pip install -qq transformers datasets

In [1]:
import transformers
import datasets
import pandas as pd
import ast

In [17]:
from datasets import load_dataset

ds = load_dataset("JanosAudran/financial-reports-sec", "small_full")

In [3]:
# SOME EDA 
# Explore the dataset structure
# print("Dataset Info:")
# print(ds)

# Access a few samples from the dataset (first 5 samples for example)
# print("\nSample Records:")
# for i in range(5):
#     print(f"Sample {i+1}:")
#     print(ds['train'][i])
#     print("\n")

# You can also look at the column names or specific attributes
print("\nColumn Names:")
print(ds['train'].column_names)



# Get dataset size details
train_size = len(ds['train'])
valid_size = len(ds['validation'])
test_size = len(ds['test'])

print(f"Training set size: {train_size} records")
print(f"Validation set size: {valid_size} records")
print(f"Test set size: {test_size} records")

# Get the number of columns
columns = ds['train'].column_names
num_columns = len(columns)

print(f"\nNumber of columns: {num_columns}")
print(f"Columns: {columns}")


Column Names:
['cik', 'sentence', 'section', 'labels', 'filingDate', 'name', 'docID', 'sentenceID', 'sentenceCount', 'tickers', 'exchanges', 'entityType', 'sic', 'stateOfIncorporation', 'tickerCount', 'acceptanceDateTime', 'form', 'reportDate', 'returns']
Training set size: 200000 records
Validation set size: 20000 records
Test set size: 20000 records

Number of columns: 19
Columns: ['cik', 'sentence', 'section', 'labels', 'filingDate', 'name', 'docID', 'sentenceID', 'sentenceCount', 'tickers', 'exchanges', 'entityType', 'sic', 'stateOfIncorporation', 'tickerCount', 'acceptanceDateTime', 'form', 'reportDate', 'returns']


In [4]:
df = ds["train"].to_pandas()
# df = datasets.concatenate_datasets([ds["train"], ds['test']]).to_pandas()

In [5]:
# size of df
len(df)

200000

In [6]:
# Unique companies
df['name'].unique(),df['name'].nunique() 

(array(['AAR CORP', 'ABBOTT LABORATORIES', 'WORLDS INC',
        'ACME UNITED CORP', 'ADAMS RESOURCES & ENERGY, INC.',
        'BK Technologies Corp', 'ADVANCED MICRO DEVICES INC',
        'Air Products & Chemicals, Inc.', 'CECO ENVIRONMENTAL CORP',
        'Matson, Inc.'], dtype=object),
 10)

In [7]:
# unique sections
df['section'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [8]:
# how many documents ? 
df['docID'].nunique()

188

In [9]:


df_sections = df.groupby(['docID', 'section']).agg({
    'sentence': ' '.join,
    'name': 'first',  # Keep the first value for this col
    'labels' : 'first', 
    'filingDate' : 'first', 
    # 'tickers' : 'first', 
    # 'stateOfIncorporation' : 'first',
    'acceptanceDateTime' : 'first',
    'reportDate' : 'first',
    'returns' : 'first'
    
}).reset_index()

In [10]:
df_sections.head()

,docID,section,sentence,name,labels,filingDate,acceptanceDateTime,reportDate,returns
0,0000001750_10-K_1994,0,ITEM 1. BUSINESS AAR CORP. and its subsidiarie...,AAR CORP,"{'1d': 1, '5d': 1, '30d': 1}",1994-08-24,1994-08-24T00:00:00.000Z,1994-05-31,{'1d': {'closePriceEndDate': 7.062620639801025...
1,0000001750_10-K_1994,3,ITEM 2. PROPERTIES Aviation trading activities...,AAR CORP,"{'1d': 1, '5d': 1, '30d': 1}",1994-08-24,1994-08-24T00:00:00.000Z,1994-05-31,{'1d': {'closePriceEndDate': 7.062620639801025...
2,0000001750_10-K_1994,4,ITEM 3. LEGAL PROCEEDINGS The Company is not a...,AAR CORP,"{'1d': 1, '5d': 1, '30d': 1}",1994-08-24,1994-08-24T00:00:00.000Z,1994-05-31,{'1d': {'closePriceEndDate': 7.062620639801025...
3,0000001750_10-K_1994,5,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,AAR CORP,"{'1d': 1, '5d': 1, '30d': 1}",1994-08-24,1994-08-24T00:00:00.000Z,1994-05-31,{'1d': {'closePriceEndDate': 7.062620639801025...
4,0000001750_10-K_1994,6,ITEM 5. MARKET FOR THE COMPANY'S COMMON EQUITY...,AAR CORP,"{'1d': 1, '5d': 1, '30d': 1}",1994-08-24,1994-08-24T00:00:00.000Z,1994-05-31,{'1d': {'closePriceEndDate': 7.062620639801025...


In [11]:
# looking at one section
df_sections.iloc[1]['sentence']

"ITEM 2. PROPERTIES Aviation trading activities are conducted from two buildings in Elk Grove Village, Illinois, one owned by the Company, the other subject to an industrial revenue bond mortgage until 1995. In addition to warehouse space, which is mechanized for efficient access to the diverse inventory, these facilities include executive offices, sales offices and a service center. Warehouse facilities are leased in Cerritos, California and Hawthorne, New York for the purpose of aviation hardware distribution and in Hamburg, Germany and Nantgarw, United Kingdom for the purpose of aviation part and component distribution. Aviation overhaul facilities are located in The Netherlands near Schiphol International Airport (owned by the Company); Garden City, New York (owned by the Company); Frankfort, New York (subject to an industrial revenue bond lease to the Company until 2001, at which time the Company shall purchase the facility for a nominal consideration); Windsor, Connecticut (in a 

In [12]:
df.columns

Index(['cik', 'sentence', 'section', 'labels', 'filingDate', 'name', 'docID',
       'sentenceID', 'sentenceCount', 'tickers', 'exchanges', 'entityType',
       'sic', 'stateOfIncorporation', 'tickerCount', 'acceptanceDateTime',
       'form', 'reportDate', 'returns'],
      dtype='object')

In [13]:
# convert the dict val inside the labels col to 3 separate cols for 1d, 5d and 30d window. 

def parse_labels(label):
    if isinstance(label, str):  # Check if the entry is a string
        try:
            return json.loads(label.replace("'", '"'))  # Convert single quotes to double quotes for JSON
        except json.JSONDecodeError as e:
            print(f"Error parsing label: {label} -> {e}")
            return {}  # Return an empty dict if parsing fails
    return label 


labels_df = df_sections['labels'].apply(parse_labels).apply(pd.Series)
labels_df.columns = [f"label_{col}" for col in labels_df.columns]
df_final = pd.concat([df_sections, labels_df], axis=1).drop(columns=['labels'])
df_final.rename(columns={'sentence': 'text'}, inplace=True)


In [14]:
df_final.head()

,docID,section,text,name,filingDate,acceptanceDateTime,reportDate,returns,label_1d,label_5d,label_30d
0,0000001750_10-K_1994,0,ITEM 1. BUSINESS AAR CORP. and its subsidiarie...,AAR CORP,1994-08-24,1994-08-24T00:00:00.000Z,1994-05-31,{'1d': {'closePriceEndDate': 7.062620639801025...,1,1,1
1,0000001750_10-K_1994,3,ITEM 2. PROPERTIES Aviation trading activities...,AAR CORP,1994-08-24,1994-08-24T00:00:00.000Z,1994-05-31,{'1d': {'closePriceEndDate': 7.062620639801025...,1,1,1
2,0000001750_10-K_1994,4,ITEM 3. LEGAL PROCEEDINGS The Company is not a...,AAR CORP,1994-08-24,1994-08-24T00:00:00.000Z,1994-05-31,{'1d': {'closePriceEndDate': 7.062620639801025...,1,1,1
3,0000001750_10-K_1994,5,ITEM 4. SUBMISSION OF MATTERS TO A VOTE OF SEC...,AAR CORP,1994-08-24,1994-08-24T00:00:00.000Z,1994-05-31,{'1d': {'closePriceEndDate': 7.062620639801025...,1,1,1
4,0000001750_10-K_1994,6,ITEM 5. MARKET FOR THE COMPANY'S COMMON EQUITY...,AAR CORP,1994-08-24,1994-08-24T00:00:00.000Z,1994-05-31,{'1d': {'closePriceEndDate': 7.062620639801025...,1,1,1


In [15]:
filename = '10k_sections_small.csv'
df_final.to_csv(filename)